In [1]:
I = [3,0,8,1,6,7,1,3,1,9,0,1]
O = [1,1,1,1,3,3,0,0,8,6,7,9]

In [ ]:
I = [5,8,4,8,9,5,6,5]
o = [5,5,5,8,8,4,9,6]

In [2]:
from collections import Counter

In [27]:
# sum(I)

In [32]:
# count = 0
dictionary = {}
for i in I:
    count = 0
    for j in I:
        if i==j:
            count = count + 1
            dictionary[i] = count
        else:
            pass
            
#     dictionary.key = i
#     dictionary[i] = count 
# print(count)
print(dictionary)    

{3: 2, 0: 2, 8: 1, 1: 4, 6: 1, 7: 1, 9: 1}


In [32]:
# count = 0
dictionary = {}
for i in I:
    count = 0
    for j in I:
        if i==j:
            count = count + 1
            dictionary[i] = count
        else:
            pass
        
        
#     dictionary.key = i
#     dictionary[i] = count 
# print(count)
print(dictionary)    

{3: 2, 0: 2, 8: 1, 1: 4, 6: 1, 7: 1, 9: 1}


In [51]:
dictionary.keys()

dict_keys([3, 0, 8, 1, 6, 7, 9])

In [53]:
dictionary.items()

dict_items([(3, 2), (0, 2), (8, 1), (1, 4), (6, 1), (7, 1), (9, 1)])

In [69]:
dict2 = {}
for i,j in dictionary.items():
    for k,l in dictionary.items():
        if j>l:
            dict2[i] = j  
        else: 
            dict2[i] = l

In [79]:
dictionary.values()

dict_values([2, 2, 1, 4, 1, 1, 1])

In [73]:
dict3=[]
for i in dictionary.values():
    for k in dictionary.values():
        if i>k:
            dict3.append(i)  
        else: 
            pass

In [74]:
dict3

[2, 2, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 4, 4]

In [70]:
dict2

{3: 1, 0: 1, 8: 1, 1: 1, 6: 1, 7: 1, 9: 1}

In [60]:
dictionary

{3: 2, 0: 2, 8: 1, 1: 4, 6: 1, 7: 1, 9: 1}

In [54]:
for i,j in dictionary.items():
#     print(i,j)
    if j<j+1:
        

3 2
0 2
8 1
1 4
6 1
7 1
9 1


In [ ]:
def func(input1):
    c = Counter(input1)
    for i in c:
        